In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import StringType
from pyspark import SQLContext
import numpy as np

conf = SparkConf().setMaster("local").setAppName("Mi programa")
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

dfspark = sqlContext.read.format("csv").option("header", "true").option("inferSchema","true").load("C:/Users/Gabo0/Desktop/Jupyter Python/Ejercicios Linkedin/base_datos_2008.csv")
dfspark = dfspark.sample(fraction = 0.001, withReplacement=False)
dfspark = dfspark.withColumn("ArrDelay",dfspark["ArrDelay"].cast("integer"))

df2 = dfspark.na.drop(subset=['ArrDelay','DepDelay','Distance'])
df2 = df2.filter("ArrDelay is not NULL")
df2 = df2.dropDuplicates()

In [2]:
# (select)Selecciona los valores donde ArrDelay es > 60
df2.select("ArrDelay").filter("ArrDelay > 60").take(5)

[Row(ArrDelay=114),
 Row(ArrDelay=191),
 Row(ArrDelay=146),
 Row(ArrDelay=95),
 Row(ArrDelay=62)]

In [3]:
# Devuelve toda la información de la fila donde ArrDelay cumple la condición
df2.filter("ArrDelay > 60").take(5)

[Row(Year=2008, Month=6, DayofMonth=23, DayOfWeek=1, DepTime='1204', CRSDepTime=1023, ArrTime='1323', CRSArrTime=1129, UniqueCarrier='OO', FlightNum=5494, TailNum='N560SW', ActualElapsedTime='79', CRSElapsedTime='66', AirTime='41', ArrDelay=114, DepDelay='101', Origin='SFO', Dest='RDD', Distance=199, TaxiIn='11', TaxiOut='27', Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay='0', WeatherDelay='0', NASDelay='13', SecurityDelay='0', LateAircraftDelay='101'),
 Row(Year=2008, Month=1, DayofMonth=31, DayOfWeek=4, DepTime='1152', CRSDepTime=900, ArrTime='1811', CRSArrTime=1500, UniqueCarrier='AA', FlightNum=1238, TailNum='N549AA', ActualElapsedTime='259', CRSElapsedTime='240', AirTime='225', ArrDelay=191, DepDelay='172', Origin='SEA', Dest='ORD', Distance=1721, TaxiIn='13', TaxiOut='21', Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay='0', WeatherDelay='0', NASDelay='191', SecurityDelay='0', LateAircraftDelay='0'),
 Row(Year=2008, Month=2, DayofMonth=19, DayOfWeek=2,

In [4]:
media = np.mean(df2.select('ArrDelay').collect())
# para aplicar una función a una columna se debe hacer con .rdd.map
df2.select("ArrDelay").rdd.map(lambda x: (x - media)**2).take(10)

[array([5.32810841]),
 array([187.46348216]),
 array([372.8092738]),
 array([215.84694302]),
 array([106.26042153]),
 array([858.97466521]),
 array([335.19273466]),
 array([69.02734325]),
 array([640.50850864]),
 array([980.20774349])]

In [5]:
# la columna que queremos mas la función descriptiva
df2.groupby("DayOfWeek").count().show()

+---------+-----+
|DayOfWeek|count|
+---------+-----+
|        1|  977|
|        6|  817|
|        3| 1018|
|        5| 1006|
|        4| 1051|
|        7|  961|
|        2|  966|
+---------+-----+



In [6]:
# Resumen, columna que queremos y la media ejemplo
df2.groupBy("DayOfWeek").mean("ArrDelay").show()

+---------+------------------+
|DayOfWeek|     avg(ArrDelay)|
+---------+------------------+
|        1| 6.487205731832139|
|        6| 5.657282741738066|
|        3| 5.944007858546169|
|        5|12.294234592445328|
|        4| 7.799238820171265|
|        7|  8.76690946930281|
|        2|10.830227743271221|
+---------+------------------+



In [7]:
#Ver todos los origenes distintos que queremos ver
# en nuestra base de datos
df2.select("Origin").rdd.distinct().take(5)

[Row(Origin='PIT'),
 Row(Origin='FSM'),
 Row(Origin='SMF'),
 Row(Origin='SPI'),
 Row(Origin='CWA')]

In [8]:
df2.select("Origin").rdd.distinct().count()

253

In [ ]:
df2.orderBy(df2.ArrDelay.desc()).take(5)